# Mahjong Classification

This notebook aim on aply transfer learning on vgg16 to Mahjong detection.

In [1]:
# import dependency

import os
import math
import h5py
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard, LearningRateScheduler
import keras
import datetime
import time
import sys
from pathlib import Path

Using TensorFlow backend.


In [2]:
# dimensions of our images.
#img_width, img_height = 224, 224
#as most of our test pattern around 40*40 , lets try some smaller size
#reset as no batter performace in 43 classes
img_width, img_height = 80, 80 

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
test_data_dir = 'data/test/'
nb_train_samples = 10800
nb_validation_samples = 450
#nb_test_samples = 550

#nb_epoch = 100
#it seems it cound learn more
nb_epoch = 35
#nb_class = 34
# update for classify 42 pattern and backOnly
nb_class = 43

ts = time.time()
logName = datetime.datetime.fromtimestamp(ts).strftime('%Y_%m_%d_%H_%M_%S')

In [3]:

# build the VGG16 network 'imagenet'
model = Sequential()
vggLayer = VGG16(include_top=False, weights=None, input_shape=(img_width, img_height, 3))


model.add(vggLayer)
model.add(Flatten(input_shape=model.output_shape[1:]))
model.add(Dense(2000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(output_dim=nb_class, activation='softmax', name='fc2'))

In [4]:
#load the weight 
# load weights into new model
model.load_weights("weights/2017_03_10_22_47_14/weights-improvement-20-0.95.hdf5")

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

test_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=8,
        class_mode='categorical')



# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.25
    drop = 0.25
    epochs_drop = 20.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

lrate = LearningRateScheduler(step_decay)

tfBoard = keras.callbacks.TensorBoard(log_dir='./logs/'+logName+"/", histogram_freq=2, write_graph=True)
filepath="./weights/"+logName+"/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')


callbacks_list = [checkpoint, tfBoard, lrate]


if not os.path.exists("./weights/"):
    os.makedirs("./weights/")   
if not os.path.exists('./logs/'):
    os.makedirs('./logs/')   
if not os.path.exists("./weights/"+logName):
    os.makedirs("./weights/"+logName)   
if not os.path.exists('./logs/'+logName):
    os.makedirs('./logs/'+logName)   

Found 2325 images belonging to 43 classes.


In [63]:
#code for printing the prediciton 
#predict_generator(self, generator, val_samples, max_q_size=10, nb_worker=1, pickle_safe=False)


test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(img_height, img_width),
        batch_size=8,
        class_mode='categorical',
        shuffle=False
        )


predicted = model.predict_generator(test_generator, val_samples=591)

test_pos_class_dict = dict(zip(test_generator.class_indices.values(), test_generator.class_indices.keys()) )


with open("./predictedResult.csv",'a') as predictedCsv:
    predictedCsv.write( "fileName,Org Class,Predicted Class\n" )
    for idx, _ in enumerate(predicted):
        predictedCsv.write(  str(os.path.basename(imgfilenames[idx])) +","+ str(os.path.dirname(imgfilenames[idx]))+","+ str(test_pos_class_dict.get( predicted[idx].argmax()))+'\n')


score = model.evaluate_generator(test_generator, val_samples=591)
print ("loss: %.2f%%\n acc: %.2f%%" %( score[0]*100, score[1]*100) )


Found 591 images belonging to 43 classes.
loss: 29.67%
 acc: 94.08%
